In [ ]:
import json
import random
import datetime
import re
import requests
import os

# ------------------------------
# Chatbot Class
# ------------------------------
class Chatbot:
    def __init__(self, name="PyBot", history_file="chat_history.json", user_file="user_data.json"):
        self.name = name
        self.history_file = history_file
        self.user_file = user_file
        self.history = self.load_json(self.history_file) or []
        self.user_data = self.load_json(self.user_file) or {"reminders": []}
        self.reminders = self.user_data.get("reminders", [])

        # Keyword-based responses
        self.responses = {
            "hello": ["Hi there!", "Hello!", "Hey! How can I help you?"],
            "good morning": ["Good morning! Have a great day!", "Morning! How are you today?"],
            "good afternoon": ["Good afternoon! Hope you're doing well!"],
            "good evening": ["Good evening! How was your day?"],
            "bye": ["Goodbye! Have a nice day.", "See you later!"],
            "how are you": ["I'm just a bot, but I'm doing great!", "Doing well, thanks!"],
            "joke": ["Want to hear a joke? Type 'joke'!"]
        }

    # ------------------------------
    # JSON Handling
    # ------------------------------
    def load_json(self, filename):
        try:
            if os.path.exists(filename):
                with open(filename, "r") as f:
                    return json.load(f)
            return None
        except:
            return None

    def save_json(self, filename, data):
        with open(filename, "w") as f:
            json.dump(data, f, indent=4)

    # ------------------------------
    # Chat Handling
    # ------------------------------
    def add_message(self, sender, message):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.history.append({"sender": sender, "message": message, "time": timestamp})
        self.save_json(self.history_file, self.history)

    def show_history(self):
        print("\n--- Chat History ---")
        for msg in self.history:
            print(f"[{msg['time']}] {msg['sender']}: {msg['message']}")
        print("--------------------\n")

    # ------------------------------
    # Reminder Handling
    # ------------------------------
    def add_reminder(self, text):
        reminder = {"text": text, "time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        self.reminders.append(reminder)
        self.user_data["reminders"] = self.reminders
        self.save_json(self.user_file, self.user_data)
        return f"Reminder added: {text}"

    def show_reminders(self):
        if not self.reminders:
            return "No reminders set."
        return "\n".join([f"{i+1}. {r['text']} ({r['time']})" for i, r in enumerate(self.reminders)])

    def delete_reminder(self, index):
        if 0 <= index < len(self.reminders):
            removed = self.reminders.pop(index)
            self.user_data["reminders"] = self.reminders
            self.save_json(self.user_file, self.user_data)
            return f"Deleted reminder: {removed['text']}"
        return "Invalid reminder number."

    # ------------------------------
    # Joke Feature
    # ------------------------------
    def get_joke(self):
        try:
            response = requests.get("https://official-joke-api.appspot.com/random_joke", timeout=5)
            data = response.json()
            return f"{data['setup']} ... {data['punchline']}"
        except:
            return "Sorry, I couldn't fetch a joke right now."

    # ------------------------------
    # Weather Feature
    # ------------------------------
    def get_weather(self, city="London"):
        try:
            response = requests.get(f"https://wttr.in/{city}?format=3", timeout=5)
            return response.text
        except:
            return "Sorry, I couldn't fetch the weather right now."

    # ------------------------------
    # Main Response Logic
    # ------------------------------
    def respond(self, user_input):
        user_input_lower = user_input.lower()

        # 1️⃣ Handle reminders
        if user_input_lower.startswith("reminder:"):
            reminder_text = user_input.split(":", 1)[1].strip()
            return self.add_reminder(reminder_text)
        elif user_input_lower == "show reminders":
            return self.show_reminders()
        elif user_input_lower.startswith("delete reminder"):
            parts = user_input_lower.split()
            if len(parts) == 3 and parts[2].isdigit():
                return self.delete_reminder(int(parts[2])-1)
            else:
                return "Please specify a valid reminder number. Example: delete reminder 1"

        # 2️⃣ Handle weather
        if user_input_lower.startswith("weather"):
            parts = user_input_lower.split()
            city = parts[1] if len(parts) > 1 else "London"
            return self.get_weather(city)

        # 3️⃣ Keyword responses (greetings, jokes, etc.)
        for keyword in self.responses:
            if re.search(rf"\b{keyword}\b", user_input_lower):
                reply = random.choice(self.responses[keyword])
                if keyword == "joke":
                    reply = self.get_joke()
                return reply

        # 4️⃣ Math evaluation
        if re.match(r"^[0-9+\-*/().\s]+$", user_input):
            try:
                return str(eval(user_input))
            except:
                return "I couldn't calculate that."

        # 5️⃣ Default response
        return ("Sorry, I didn’t understand that. You can say 'hello', 'good morning', "
                "'good evening', 'joke', 'reminder: <text>', 'show reminders', 'delete reminder <number>', "
                "'weather <city>', or type a math expression.")

# ------------------------------
# Main Chat Loop
# ------------------------------
def main():
    bot = Chatbot()

    print(f"💬 Welcome to {bot.name}! Type 'bye' to exit.")
    print("You can chat with me, ask for jokes, check weather, manage reminders, or do math.\n")

    while True:
        user_input = input("You: ").strip()
        if not user_input:
            continue

        bot.add_message("You", user_input)
        reply = bot.respond(user_input)
        print(f"{bot.name}: {reply}\n")
        bot.add_message(bot.name, reply)

        if "bye" in user_input.lower():
            break

    bot.show_history()

if __name__ == "__main__":
    main()


💬 Welcome to PyBot! Type 'bye' to exit.
You can chat with me, ask for jokes, check weather, manage reminders, or do math.

